Extract YASA spindle parameters for all subjects so far (sub02 - sub07) for the adaptation nights

In [1]:
import yasa
import mne
import matplotlib.pyplot as plt
import numpy as np
import statistics
import pybv

In [61]:
# some preliminaries
datapath = 'E:\\spindle_ppTMS\\EEG'

In [55]:
def get_spindle_param(subject): 
    """ reads in continous data from the adaptation night without TMS artifacts and does automatic sleep stageing and spindle detection
    using  YASA
    input subject needs to be a string """
    
    data_raw = mne.io.read_raw_edf((datapath + '\\' + subject + '\\' + 'ses-adapt' +'\\' + 'data_' + subject + '_clean_continous_resampled.edf'), preload=True, verbose=None)
    data_raw = data_raw.pick_channels(["C4", "VEOG", "EMG", "HEOG", "TP9", "TP10"], ordered=False)
    data_raw.filter(0.1, 40)

    # yasa sleep stageing
    sleep_stages = yasa.SleepStaging(data_raw, eeg_name = "C4", eog_name= "VEOG", emg_name="EMG") 
    values = sleep_stages.predict() # predict the sleep stages
    hyp = yasa.Hypnogram(values, n_stages=5) # create yasa Hypnogram object
    # hyp.plot_hypnogram()

    # yasa spindle detection 
    hyp_up = hyp.upsample_to_data(sf=1/30, data = data_raw)
    # data_c4 = data_raw.get_data(picks=['C4'], units="uV") - data_raw.get_data(picks=['TP9'], units="uV")
    data_c4 = data_raw.get_data(picks=['C4']) -  data_raw.get_data(picks=['TP9']) 
    spindle_c4 = yasa.spindles_detect(data_c4, hypno=hyp_up, sf = 100, ch_names = ['C4'], freq_sp=(10, 15)) # include N2 and N3 sleep
    spindles_c4 = spindle_c4.summary()
    print(spindles_c4)

    # get RMS and spindle frequency 
    mean_RMS = statistics.mean(spindles_c4.RMS)
    print('RMS for:', subject, mean_RMS)
    mean_SpF = statistics.mean(spindles_c4.Frequency)
    print('SpF for:', subject, mean_SpF)

    return values, hyp, spindles_c4, mean_RMS, mean_SpF

In [63]:
def get_sleep_statistics(values):

    W = 0
    N1 = 0
    N2 = 0
    N3 = 0
    R = 0

    for i in values:
        if i == 'W':
            W = W + 1
        elif i == 'N1':
            N1 = N1 + 1
        elif i == 'N2':
            N2 = N2 + 1
        elif i == 'N3':
            N3 = N3 + 1
        else:
            R = R + 1

    # get part of night were subject first sleeps
    first_N1_epoch = np.where(values == "N1")[0][0] # first row, start in first column # or np.argmax(y_pred == "N1")

    # find wake epochs that were not at start/end of recording to find last sleep epoch
    if values[-1] == 'W':
        last_sleep_epoch = (np.where(values == "W")[0][-1])-1 # first raw, start in last column
    else:
        last_sleep_epoch = len(values)
        
    values_asleep = values[first_N1_epoch:last_sleep_epoch]

    A = np.count_nonzero(values_asleep == 'W') # get arousals outside of wake time and beginning and end of recording
    print(A)


    # Calculate % of total time

    perc_W_total = int(W/(W+N1+N2+N3+R)*100)
    perc_N1_total= int(N1/(W+N1+N2+N3+R)*100)
    perc_N2_total= int(N2/(W+N1+N2+N3+R)*100)
    perc_N3_total = int(N3/(W+N1+N2+N3+R)*100)
    perc_R_total = int(R/(W+N1+N2+N3+R)*100)
    perc_stages_total = [ perc_W_total, perc_N1_total, perc_N2_total, perc_N3_total, perc_R_total]


    # Calculate % of total sleep time 

    perc_A_sleep = int(A/(A+N1+N2+N3+R)*100)
    perc_N1_sleep= int(N1/(A+N1+N2+N3+R)*100)
    perc_N2_sleep = int(N2/(A+N1+N2+N3+R)*100)
    perc_N3_sleep = int(N3/(A+N1+N2+N3+R)*100)
    perc_R_sleep = int(R/(A+N1+N2+N3+R)*100)
    perc_stages_sleep = [perc_A_sleep, perc_N1_sleep, perc_N2_sleep, perc_N3_sleep, perc_R_sleep]

    # Print results
    print('The percentage of W stage total is: ' + str(perc_W_total) + '%')
    print('The percentage of N1 stage total is: ' + str(perc_N1_total) + '%')
    print('The percentage of N2 stage total is: ' + str(perc_N2_total) + '%')
    print('The percentage of N3 stage total is: ' + str(perc_N3_total) + '%')
    print('The percentage of R stage total is: ' + str(perc_R_total) + '%')

    print('The percentage of A stage asleep is: ' + str(perc_A_sleep) + '%')
    print('The percentage of N1 stage asleep is: ' + str(perc_N1_sleep) + '%')
    print('The percentage of N2 stage asleep is: ' + str(perc_N2_sleep) + '%')
    print('The percentage of N3 stage asleep is: ' + str(perc_N3_sleep) + '%')
    print('The percentage of R stage asleep is: ' + str(perc_R_sleep) + '%')

    return perc_stages_total, perc_stages_sleep

In [64]:
# print the spindle parameters for sub-02
(values_sub02, hyp_sub02, spindles_c4_sub02, mean_RMS_sub02, mean_SpF_sub02) = get_spindle_param('sub-02') 

# get sleep statistics
(perc_stages_total_sub02, perc_stages_asleep_sub02) = get_sleep_statistics(values_sub02)


Extracting EDF parameters from E:\spindle_ppTMS\EEG\sub-02\ses-adapt\data_sub-02_clean_continous_resampled.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 600699  =      0.000 ...  6006.990 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 s)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\siann\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 0.24.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\siann\AppData\Local\Programs\Python\Python39\lib\site-packages\yasa\hypno.py:483: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return self.hypno.replace(self.mapping).astype(np.int16)
c:\Users\siann\AppData\Local\Programs\Python\Python39\lib\site-packages\yasa\hypno.py:483: FutureWarning: The behavior of Series.replace (and Da

      Start     Peak      End  Duration   Amplitude        RMS  AbsPower  \
0   4884.82  4885.08  4885.41      0.59   94.166543  18.602982  2.395819   
1   4889.24  4889.50  4889.83      0.59   88.636361  18.647852  2.393132   
2   4893.64  4893.91  4894.23      0.59  103.786597  18.524729  2.368833   
3   4898.10  4898.38  4898.71      0.61  104.393751  18.407321  2.353372   
4   4902.51  4902.77  4903.11      0.60   87.345096  18.563229  2.388292   
5   4996.20  4996.64  4997.43      1.23   65.082965  13.174409  2.079580   
6   5000.61  5001.08  5001.88      1.27   65.628434  13.084910  2.065821   
7   5004.99  5005.46  5006.26      1.27   65.559785  13.065748  2.065829   
8   5009.47  5009.91  5010.69      1.22   64.308564  13.268579  2.092566   
9   5013.89  5014.33  5015.12      1.23   66.014461  13.188362  2.076238   
10  5289.66  5290.00  5290.19      0.53   73.413222  14.570270  2.316974   
11  5294.10  5294.46  5294.65      0.55   71.642096  14.342915  2.296639   
12  5298.53 

In [67]:
# get the spindle parameters for sub-03
(values_sub03, hyp_sub03, spindles_c4_sub03, mean_RMS_sub03, mean_SpF_sub03) = get_spindle_param('sub-03') 

# get sleep statistics
(perc_stages_total_sub03, perc_stages_asleep_sub03) = get_sleep_statistics(values_sub03)

Extracting EDF parameters from E:\spindle_ppTMS\EEG\sub-03\ses-adapt\data_sub-03_clean_continous_resampled.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 143099  =      0.000 ...  1430.990 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 s)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\siann\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 0.24.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\siann\AppData\Local\Programs\Python\Python39\lib\site-packages\yasa\hypno.py:483: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return self.hypno.replace(self.mapping).astype(np.int16)
c:\Users\siann\AppData\Local\Programs\Python\Python39\lib\site-packages\yasa\hypno.py:483: FutureWarning: The behavior of Series.replace (and Da

     Start     Peak      End  Duration  Amplitude        RMS  AbsPower  \
0   172.99   173.56   174.07      1.08  60.259123  11.048151  1.917290   
1   767.19   767.41   768.14      0.95  58.185593  11.689194  2.122190   
2   785.13   785.67   785.71      0.58  39.853414   8.981418  1.923213   
3   861.83   862.10   862.38      0.55  49.459386  11.821889  2.190269   
4  1298.17  1298.49  1298.91      0.74  44.473651  11.382232  2.097942   
5  1309.79  1309.96  1310.57      0.78  39.607358   9.131289  2.114981   

   RelPower  Frequency  Oscillations  Symmetry  Stage Channel  IdxChannel  
0  0.330009  12.509190          14.0  0.522936      3      C4           0  
1  0.447776  12.457001          12.0  0.229167      2      C4           0  
2  0.382616  12.651841           7.0  0.915254      2      C4           0  
3  0.253933  11.215726           5.0  0.482143      1      C4           0  
4  0.404584  12.121269          10.0  0.426667      2      C4           0  
5  0.359489  12.834602   